In [1]:
from importlib import reload
import xarray as xr
import dominosee.es as es
import dominosee.network as net

In [2]:
import dask.config
dask.config.set(scheduler='processes')

# Event selection

In [ ]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .isel(lat=slice(0, 25), lon=slice(0, 20), time=slice(0, 2000)).load()
ds_stack = ds.stack(location=["lat", "lon"])
ds

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 10957, lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lon      (lon) float64 3kB 60.0 60.25 60.5 60.75 ... 149.0 149.2 149.5 149.8
    number   int64 8B ...
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
Data variables:
    SPI1     (time, lat, lon) float32 3GB -0.5392 -0.631 -0.7163 ... 1.324 1.348

In [33]:
reload(es)
event_ds = es.get_event_positions(ds.SPI1 < -1.5)
event_ds_stack = es.get_event_positions(ds_stack.SPI1 < -1.5)
event_ds

<xarray.Dataset> Size: 753kB
Dimensions:          (lon: 20, lat: 25, event: 373)
Coordinates:
  * lon              (lon) float64 160B 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * lat              (lat) float64 200B 54.25 54.0 53.75 ... 48.75 48.5 48.25
  * event            (event) int64 3kB 0 1 2 3 4 5 6 ... 367 368 369 370 371 372
Data variables:
    event_positions  (lat, lon, event) int32 746kB 37 38 39 40 ... -1 -1 -1 -1
    event_count      (lat, lon) int64 4kB 267 267 270 267 ... 175 162 116 115

In [34]:
from dominosee import eventorize

In [41]:
reload(eventorize)
da_ep = eventorize.select_first_consecutive(ds.SPI1 < -1.5, period=1)
da_ep_stack = eventorize.select_first_consecutive(ds_stack.SPI1 < -1.5, period=1)
da_ep

<xarray.DataArray 'SPI1' (lat: 25, lon: 20, time: 2000)> Size: 1MB
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False,  True],
        [ True, False, False, ..., False, False,  True]],

       [[ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False]],

       [[ True, False, False, ...,  True, False, False],
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ...,  True, False,  True],
        [False, False, False, ..., False, False, False]]],
      shape=(25, 20, 2000))
Coordinates:
  * time     (time) datetime64[ns] 16kB 1950-01-01 1950-01-02 ... 1955-06-23
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25

In [42]:
burst_ts = es.get_event_positions(da_ep)
burst_ts_stack = es.get_event_positions(da_ep_stack)
burst_ts

<xarray.Dataset> Size: 97kB
Dimensions:          (lon: 20, lat: 25, event: 46)
Coordinates:
  * lon              (lon) float64 160B 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * lat              (lat) float64 200B 54.25 54.0 53.75 ... 48.75 48.5 48.25
  * event            (event) int64 368B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45
Data variables:
    event_positions  (lat, lon, event) int32 92kB 37 292 334 352 ... -1 -1 -1 -1
    event_count      (lat, lon) int64 4kB 30 28 28 28 31 33 ... 27 27 20 21 21

# Calculate ES on unstacked and stacked data

In [43]:
burst_ts_diff = es.get_event_time_differences(burst_ts.event_positions, burst_ts.event_count)
burst_ts_diff_stack = es.get_event_time_differences(burst_ts_stack.event_positions, burst_ts_stack.event_count)
burst_ts_diff

<xarray.DataArray 'event_positions' (lat: 25, lon: 20, event: 46)> Size: 184kB
array([[[255.,  42.,  18., ...,  nan,  nan,  nan],
        [254.,  43.,  18., ...,  nan,  nan,  nan],
        [254.,  43.,  18., ...,  nan,  nan,  nan],
        ...,
        [ 22.,  32.,  75., ...,  nan,  nan,  nan],
        [ 21.,  32.,  75., ...,  nan,  nan,  nan],
        [  6.,   3.,  12., ...,  nan,  nan,  nan]],

       [[ 59., 194.,  40., ...,  nan,  nan,  nan],
        [ 59., 194.,  40., ...,  nan,  nan,  nan],
        [ 59., 194.,  43., ...,  nan,  nan,  nan],
        ...,
        [  6.,   3.,  12., ...,  nan,  nan,  nan],
        [ 18.,  32., 202., ...,  nan,  nan,  nan],
        [202.,  43.,  37., ...,  nan,  nan,  nan]],

       [[ 58., 195.,  39., ...,  nan,  nan,  nan],
        [ 59., 194.,  37., ...,  nan,  nan,  nan],
        [ 59., 195.,  39., ...,  nan,  nan,  nan],
        ...,
...
        ...,
        [179.,   8.,  44., ...,  nan,  nan,  nan],
        [179.,   8.,   9., ...,  nan,  nan,  nan],
        [180.,   8.,   8., ...,  nan,  nan,  nan]],

       [[ 65.,  32.,   6., ...,  nan,  nan,  nan],
        [ 65.,  32.,  63., ...,  nan,  nan,  nan],
        [ 65.,  32.,  32., ...,  nan,  nan,  nan],
        ...,
        [179.,   9.,   9., ...,  nan,  nan,  nan],
        [187.,  16.,  19., ...,  nan,  nan,  nan],
        [180.,   7.,  16., ...,  nan,  nan,  nan]],

       [[ 97.,  56.,  26., ...,  nan,  nan,  nan],
        [ 97.,  64.,  13., ...,  nan,  nan,  nan],
        [ 97.,  23.,  60., ...,  nan,  nan,  nan],
        ...,
        [179.,  35.,   3., ...,  nan,  nan,  nan],
        [189.,  14.,  20., ...,  nan,  nan,  nan],
        [176.,  10.,  34., ...,  nan,  nan,  nan]]], shape=(25, 20, 46))
Coordinates:
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
  * event    (event) int64 368B 0 1 2 3 4 5 6 7 8 ... 37 38 39 40 41 42 43 44 45
Attributes:
    long_name:    Event Time Differences
    units:        time steps
    description:  Time differences between consecutive events for each locati...

In [58]:
reload(es)
es_ds = es.get_event_sync_from_positions(burst_ts.event_positions, burst_ts.event_positions.copy(), 
                                 tm=30)
es_ds_stack = es.get_event_sync_from_positions(burst_ts_stack.event_positions, burst_ts_stack.event_positions.copy(), 
                                 tm=30)
es_ds_stack

<xarray.DataArray 'event_positions' (locationA: 500, locationB: 500)> Size: 250kB
array([[28, 25, 24, ...,  7,  6,  7],
       [25, 26, 25, ...,  7,  6,  7],
       [24, 25, 26, ...,  7,  6,  6],
       ...,
       [ 7,  7,  7, ..., 18, 13, 12],
       [ 7,  7,  7, ..., 14, 19, 15],
       [ 8,  8,  7, ..., 13, 15, 19]], shape=(500, 500), dtype=uint8)
Coordinates:
  * locationA  (locationA) object 4kB MultiIndex
  * latA       (locationA) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonA       (locationA) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * locationB  (locationB) object 4kB MultiIndex
  * latB       (locationB) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonB       (locationB) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    tau_max:       30
    output_dtype:  <class 'numpy.uint8'>

In [56]:
assert es_ds.stack(locationA=["latA", "lonA"], locationB=["latB", "lonB"]).equals(es_ds_stack)

# Null model

In [73]:
reload(es)
da_cvalues = es.create_null_model_from_indices(ds.time, es_ds.attrs["tau_max"], 
                                               max_events=burst_ts.event_count.max().values, significances=[0.05, 0.01, 0.005], min_es=2)
da_cvalues

<xarray.DataArray (noeA: 47, noeB: 47, significance: 3)> Size: 53kB
array([[[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        ...,
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        ...,
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        ...,
...
        ...,
        [4, 6, 7],
        [4, 6, 7],
        [4, 6, 6]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        ...,
        [4, 6, 7],
        [4, 6, 7],
        [4, 6, 7]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        ...,
        [4, 6, 6],
        [4, 6, 7],
        [4, 6, 6]]], shape=(47, 47, 3))
Coordinates:
  * noeA          (noeA) int64 376B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45 46
  * noeB          (noeB) int64 376B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45 46
  * significance  (significance) float64 24B 0.05 0.01 0.005
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      30
    max_events:   46
    min_es:       2

In [74]:
temp = da_cvalues.sel(significance=0.005).squeeze()
temp

<xarray.DataArray (noeA: 47, noeB: 47)> Size: 18kB
array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [2, 2, 2, ..., 7, 7, 6],
       [2, 2, 2, ..., 7, 7, 7],
       [2, 2, 2, ..., 6, 7, 6]], shape=(47, 47))
Coordinates:
  * noeA          (noeA) int64 376B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45 46
  * noeB          (noeB) int64 376B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45 46
    significance  float64 8B 0.005
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      30
    max_events:   46
    min_es:       2

In [75]:
reload(es)
da_null = es.convert_null_model_for_locations(da_cvalues, burst_ts.event_count, burst_ts.event_count, sig=0.005)
da_null

<xarray.DataArray (latA: 25, lonA: 20, latB: 25, lonB: 20)> Size: 2MB
array([[[[4, 4, 4, ..., 5, 4, 5],
         [5, 4, 4, ..., 5, 5, 4],
         [4, 4, 4, ..., 4, 4, 4],
         ...,
         [3, 4, 4, ..., 4, 4, 4],
         [3, 3, 4, ..., 4, 3, 4],
         [4, 4, 4, ..., 3, 3, 3]],

        [[4, 3, 3, ..., 4, 4, 5],
         [4, 4, 4, ..., 5, 5, 4],
         [4, 4, 4, ..., 4, 3, 4],
         ...,
         [3, 4, 3, ..., 3, 3, 3],
         [3, 3, 4, ..., 4, 3, 4],
         [4, 4, 3, ..., 3, 3, 3]],

        [[4, 3, 3, ..., 4, 4, 5],
         [4, 4, 4, ..., 5, 5, 4],
         [4, 4, 4, ..., 4, 3, 4],
         ...,
...
         ...,
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 4, ..., 3, 2, 3],
         [4, 4, 3, ..., 2, 2, 2]],

        [[3, 3, 3, ..., 3, 4, 4],
         [3, 3, 3, ..., 4, 4, 4],
         [3, 3, 3, ..., 3, 3, 3],
         ...,
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 2, 3, 3]],

        [[3, 3, 3, ..., 3, 4, 4],
         [3, 3, 3, ..., 4, 4, 4],
         [3, 3, 3, ..., 3, 3, 3],
         ...,
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 2, 3, 3]]]], shape=(25, 20, 25, 20))
Coordinates:
    noeA          (latA, lonA) int64 4kB 30 28 28 28 31 33 ... 26 27 27 20 21 21
    noeB          (latB, lonB) int64 4kB 30 28 28 28 31 33 ... 26 27 27 20 21 21
    significance  float64 8B 0.005
  * lonA          (lonA) float64 160B 73.0 73.25 73.5 73.75 ... 77.25 77.5 77.75
  * latA          (latA) float64 200B 54.25 54.0 53.75 53.5 ... 48.75 48.5 48.25
  * lonB          (lonB) float64 160B 73.0 73.25 73.5 73.75 ... 77.25 77.5 77.75
  * latB          (latB) float64 200B 54.25 54.0 53.75 53.5 ... 48.75 48.5 48.25
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      30
    max_events:   46
    min_es:       2

# get link

In [76]:
es_ds

<xarray.DataArray 'event_positions' (latA: 25, latB: 25, lonA: 20, lonB: 20)> Size: 250kB
array([[[[28, 25, 24, ..., 16, 13, 11],
         [25, 26, 25, ..., 15, 13, 10],
         [24, 25, 26, ..., 15, 14, 11],
         ...,
         [16, 15, 15, ..., 32, 28, 26],
         [13, 13, 14, ..., 28, 34, 29],
         [11, 10, 11, ..., 26, 29, 35]],

        [[22, 23, 22, ..., 17, 13, 12],
         [21, 22, 21, ..., 16, 13, 13],
         [21, 22, 21, ..., 17, 14, 14],
         ...,
         [15, 16, 16, ..., 27, 26, 23],
         [12, 14, 14, ..., 28, 28, 25],
         [11, 12, 12, ..., 30, 27, 25]],

        [[17, 17, 18, ..., 12, 12, 12],
         [18, 16, 17, ..., 10, 11, 10],
         [18, 16, 17, ..., 11, 12, 11],
         ...,
...
         [ 7,  8,  9, ..., 10, 13, 14],
         [ 8,  7,  8, ..., 11, 12, 13],
         [ 8,  7,  6, ..., 11, 11, 12]],

        [[18, 18, 24, ..., 13, 10, 12],
         [18, 20, 24, ..., 14, 10, 11],
         [16, 17, 20, ..., 10, 10, 12],
         ...,
         [ 8,  9, 11, ..., 14, 12, 14],
         [ 7,  8, 10, ..., 14, 14, 15],
         [ 7,  7,  9, ..., 13, 12, 14]],

        [[29, 25, 21, ..., 11, 10, 10],
         [24, 31, 18, ..., 12, 12, 11],
         [21, 19, 26, ..., 12,  9,  9],
         ...,
         [10, 11, 11, ..., 18, 13, 12],
         [11, 13,  9, ..., 14, 19, 15],
         [10, 12,  9, ..., 13, 15, 19]]]],
      shape=(25, 25, 20, 20), dtype=uint8)
Coordinates:
  * lonA     (lonA) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * latA     (latA) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
  * lonB     (lonB) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * latB     (latB) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    tau_max:       30
    output_dtype:  <class 'numpy.uint8'>

In [77]:
reload(net)
da_link = net.get_link_from_critical_values(es_ds, da_null, rule="greater")
da_link

<xarray.DataArray (latA: 25, latB: 25, lonA: 20, lonB: 20)> Size: 250kB
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
...
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]]]],
      shape=(25, 25, 20, 20))
Coordinates:
  * lonA          (lonA) float64 160B 73.0 73.25 73.5 73.75 ... 77.25 77.5 77.75
  * latA          (latA) float64 200B 54.25 54.0 53.75 53.5 ... 48.75 48.5 48.25
  * lonB          (lonB) float64 160B 73.0 73.25 73.5 73.75 ... 77.25 77.5 77.75
  * latB          (latB) float64 200B 54.25 54.0 53.75 53.5 ... 48.75 48.5 48.25
    noeA          (latA, lonA) int64 4kB 30 28 28 28 31 33 ... 26 27 27 20 21 21
    noeB          (latB, lonB) int64 4kB 30 28 28 28 31 33 ... 26 27 27 20 21 21
    significance  float64 8B 0.005

In [79]:
da_link.sum()/da_link.size

<xarray.DataArray ()> Size: 8B
array(0.993472)
Coordinates:
    significance  float64 8B 0.005